## SCENIC program template
""" Scenario Description
Traffic Scenario 01.
Control loss without previous action.
The ego-vehicle loses control due to bad conditions on the road and it must recover, coming back to
its original lane.
"""
param map = localPath('../../tests/formats/opendrive/maps/CARLA/Town01.xodr')  # or other CARLA map that definitely works

param carla_map = 'Town01'

model scenic.simulators.carla.model

EGO_MODEL = "vehicle.lincoln.mkz_2017"

EGO_SPEED = 10


behavior EgoBehavior(speed=10):
    do FollowLaneBehavior(speed)

lane = Uniform(*network.lanes)

ego = Car at (100, -330),
    with blueprint EGO_MODEL

car1 = Car offset by (Range(-20,15), Range(1,20)),
		with viewAngle Range(-20,20) deg
        
car2 = Car offset by (Range(-20,15), Range(1,20)),
		with viewAngle Range(-20,20) deg
        
car3 = Car offset by (Range(-20,15), Range(1,20)),
		with viewAngle Range(-20,20) deg

require ego can see car1

require ego can see car2

require ego can see car3

terminate when 10 > 100

# Scenario Data Generation

In [ ]:
from subprocess import run
import carla
import random
import time
import numpy as np  # numpy
import cv2          # opencv-python
import pandas as pd
import scenic


carnum = 3 # can choose 2-6
roadtype = "I"# can choose I X R T




def get_image_point(loc, K, w2c):
        # Calculate 2D projection of 3D coordinate

        # Format the input coordinate (loc is a carla.Position object)
        point = np.array([loc.x, loc.y, loc.z, 1])
        # transform to camera coordinates
        point_camera = np.dot(w2c, point)

        # New we must change from UE4's coordinate system to an "standard"
        # (x, y ,z) -> (y, -z, x)
        # and we remove the fourth componebonent also
        point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

        # now project 3D->2D using the camera matrix
        point_img = np.dot(K, point_camera)
        # normalize
        point_img[0] /= point_img[2]
        point_img[1] /= point_img[2]

        return point_img[0:2]
    
def build_projection_matrix(w, h, fov):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

for i in range(10):    
    scenic_carla = "scenic ./Scenic-master/examples/carla/basei_3.scenic --simulate  --count 1 --time 1"
    run(scenic_carla)

    actor_list = []
    npclist = []

    filename = "gzdata_s.csv"
    gzdata = pd.read_csv(filename, index_col="id")

    try:
        client = carla.Client('127.0.0.1', 2000)
        client.set_timeout(2.0)
        world = client.get_world()
        camerapos = ""
        try:
            weather = carla.WeatherParameters(
                sun_altitude_angle=random.randint(0,90),
                cloudiness=random.randint(0,100),
                precipitation=random.randint(0,100),
                precipitation_deposits = random.randint(0,100),
                wind_intensity = random.randint(0,100),
                fog_density = random.randint(0,100),
                fog_distance = random.randint(0,100),
                wetness = random.randint(0,100),
                
            )
            world.set_weather(weather)
            wea = world.get_weather()      
            aclist = world.get_actors()
            veclist = aclist.filter("vehicle*")
            if len(veclist) < 2:
                continue
            for car in veclist:
                if car.type_id == 'vehicle.lincoln.mkz_2017' and abs(car.get_transform().location.x - 100) < 5.0 and abs(car.get_transform().location.y - 330)<5.0: 
                    ego = car
                else:
                    npclist.append(car)
                actor_list.append(car)
            ego_t = ego.get_transform()
            car1_t = npclist[0].get_transform()

            egot_location = ego_t.location
            car1t_location = car1_t.location

            distance = int(egot_location.distance(car1t_location))

            ctime = wea.sun_altitude_angle # str((wea.sun_altitude_angle+90)*12/180.0)
            cloudiness = str(wea.cloudiness)
            precipitation = str(wea.precipitation)
            ponding = str(wea.precipitation_deposits)
            wind = str(wea.wind_intensity)
            fogden = str(wea.fog_density)
            fogdis = str(wea.fog_distance)
            wetness = str(wea.wetness)
            
            carcolorred = npclist[0].attributes["color"].split(',')[0]
            carcolorgreen = npclist[0].attributes["color"].split(',')[1]
            carcolorblue = npclist[0].attributes["color"].split(',')[2]
            cartype = str(npclist[0].type_id)
            pitch = str(npclist[0].get_transform().rotation.pitch)
            yaw = str(npclist[0].get_transform().rotation.yaw)
            roll = str(npclist[0].get_transform().rotation.roll)
            cardistance = str(distance)
            carnumber = str(carnum)

            # ! sensor.camera.rgb
            # Find the blueprint of the sensor
            camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
            # Modify the attributes of the blueprint to set image resolution and field of view
            camera_bp.set_attribute('image_size_x', '1920')
            camera_bp.set_attribute('image_size_y', '1080')
            camera_bp.set_attribute('fov', '110')
            # Set the time in seconds between sensor captures
            camera_bp.set_attribute('sensor_tick', '1.0')
            camera_transform = carla.Transform(carla.Location(x=0.8, z=1.7))
            sensor = world.spawn_actor(camera_bp, camera_transform, attach_to=ego)


            # do_something() will be call each time a new image is generated by the camera
            # sensor.listen(lambda data: do_something(date))
            sensor.listen(lambda image: image.save_to_disk('./outputpng_s/%03d.png' % len(gzdata)))
            time.sleep(1)
            sensor.stop()


            world_2_camera = np.array(ego.get_transform().get_inverse_matrix())
            # Get the attributes from the camera
            image_w = camera_bp.get_attribute("image_size_x").as_int()
            image_h = camera_bp.get_attribute("image_size_y").as_int()
            fov = camera_bp.get_attribute("fov").as_float()
            K = build_projection_matrix(image_w, image_h, fov)
            # 3D -> 2D 
            bb = npclist[0].bounding_box
            verts = [v for v in bb.get_world_vertices(npclist[0].get_transform())]
            x_max = -10000
            x_min = 10000
            y_max = -10000
            y_min = 10000
            for vert in verts:
                p = get_image_point(vert, K, world_2_camera)
                # Find the rightmost vertex
                if p[0] > x_max:
                    x_max = p[0]
                # Find the leftmost vertex
                if p[0] < x_min:
                    x_min = p[0]
                # Find the highest vertex
                if p[1] > y_max:
                    y_max = p[1]
                # Find the lowest  vertex
                if p[1] < y_min:
                    y_min = p[1]
            camerapos = str(int(x_min))+ ',' + str(int(y_min)) + ',' + str(int(x_max))+ ',' + str(int(y_max)) + '\n'
            gzdata.loc[len(gzdata)] = [ctime,cloudiness,precipitation,ponding,wind,fogden,fogdis,wetness,carcolorred,carcolorgreen,carcolorblue,cartype,pitch,yaw,roll,cardistance,carnumber,roadtype,camerapos] 
            gzdata.to_csv(filename)
        finally:
            print(gzdata.tail(3))
    finally:
        print('destroying actors')
        for actor in actor_list:
            actor.destroy()
        print('done.')